# Segment and quantify puncta

Segment puncta and analyze their properties. Calculate puncta summaries per ROI (cell or nucleus) if the ROI segmentation is available.


## Documentation

[Details of the puncta segmentation procedure](../docs/puncta_segmentation.md)

[Table summarizing all parameters that are used in this notebook](../docs/parameters.md#list-of-parameters-for-puncta-segmentation-and-analysis)


## Requirements

- A folder with images to analyze 
- A json parameter file specified using [setup_puncta_analysis.ipynb](setup_puncta_analysis.ipynb)


<hr style="height:2px;">

## Config

<hr style="height:2px;">

### The following code imports and declares functions used for the processing:

In [1]:
#################################
#  Don't modify the code below  #
#################################

import json
import os
from punctatools.lib.segment import segment_puncta_batch
from punctatools.lib.quantify import quantify_batch

2022-06-16 13:02:35,752 [INFO] WRITING LOG OUTPUT TO /home/amedyukh/.cellpose/run.log


### Specify the parameters for segmentation

<hr style="height:0.5px;">

`parameter_file`: json parameter file generated using [setup_puncta_analysis.ipynb](setup_puncta_analysis.ipynb).

`n_jobs`: number of processes to run in parallel. Set according to your workstation resources. Decrease if the processing runs out of memory. (Default is 8)

`input_dir`: folder with images to analyze; set to `None` to load this value from the parameter file

`output_dir`: folder to save results; set to `None` to load this value from the parameter file


In [2]:
parameter_file = '../example_data/parameters/parameters_puncta.json'
n_jobs = 8

input_dir = None
output_dir = None

<hr style="height:2px;">

## Processing

<hr style="height:2px;">


### The following code loads the parameters 

In [3]:
#################################
#  Don't modify the code below  #
#################################

with open(parameter_file) as f:
    params = json.load(f)

params['n_jobs'] = n_jobs
if input_dir is not None:
    params['input_dir'] = input_dir.rstrip('/')
if output_dir is not None:
    params['output_dir'] = output_dir.rstrip('/')
    
os.makedirs(params['output_dir'], exist_ok=True)

with open(os.path.join(params['output_dir'], parameter_file.split('/')[-1]), 'w') as f:
    json.dump(params, f, indent=4)
    
params

{'input_dir': '../example_data/stacks',
 'output_dir': '../test_output/puncta_analysis',
 'roi_segmentation': False,
 'puncta_segm_dir': 'puncta_segm',
 'puncta_quant_dir': 'puncta_quants',
 'roi_quant_dir': 'cell_quants',
 'puncta_channels': [1, 2],
 'channel_names': ['ch0', 'ch1', 'ch3'],
 'minsize_um': 0.2,
 'maxsize_um': 2,
 'num_sigma': 5,
 'overlap': 1,
 'threshold_detection': [0.001, 0.0001],
 'threshold_background': [2, 1.5],
 'global_background': False,
 'global_background_percentile': 95,
 'background_percentile': 50,
 'threshold_segmentation': [0.001, 0.0003],
 'segmentation_mode': 0,
 'remove_out_of_roi': False,
 'maxrad_um': None,
 'n_jobs': 8}

### The following code segments puncta in all image in the input folder

In [4]:
#################################
#  Don't modify the code below  #
#################################

channel_names = params.pop('channel_names')
puncta_segm_dir = params.pop('puncta_segm_dir')
roi_quant_dir = params.pop('roi_quant_dir')
puncta_quant_dir = params.pop('puncta_quant_dir')

segm_kwargs = params.copy()
segm_kwargs['output_dir'] = os.path.join(segm_kwargs['output_dir'], puncta_segm_dir)
segment_puncta_batch(parallel=True, process_name='Segment puncta', **segm_kwargs)

Run Segment puncta


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 85.79it/s]


Segment puncta done


### The following code quantifies all images

This will generate quantification for [individual puncta](../docs/outputs.md#measurements-for-individual-puncta) and [individual ROI](../docs/outputs.md#measurements-for-individual-roi).

In [5]:
#################################
#  Don't modify the code below  #
#################################

quantify_batch(input_dir=segm_kwargs['output_dir'],
               output_dir_puncta=os.path.join(params['output_dir'], puncta_quant_dir),
               output_dir_roi=os.path.join(params['output_dir'], roi_quant_dir),
               parallel=True, n_jobs=params['n_jobs'],
               channel_names=channel_names,
               puncta_channels=params['puncta_channels'],
               process_name='Quantify puncta')

Run Quantify puncta


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 97.32it/s]
/research/sharedresources/cbi/public/conda_envs/punctatools/lib/python3.9/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


Quantify puncta done


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 242.28it/s]
